In [42]:
import sqlite3
 

In [43]:
def run_query(query):
    result = None
    try:
        # Connect to DB and create a cursor
        sqlite_connection = sqlite3.connect('db.sqlite3')
        cursor = sqlite_connection.cursor()
        cursor.execute(query) 
        # Fetch and output result
        result = cursor.fetchall()
        # Close the cursor
        cursor.close()
        
    # Handle errors
    except sqlite3.Error as error:
        print('Error occurred - ', error)
 
    # Close DB Connection irrespective of success
    # or failure
    finally:
        if sqlite_connection:
            sqlite_connection.close()
    return result

In [47]:
query_1 = '''
    SELECT
        count(turns.id) as number_of_turns,
        turns.game_id as game_id,
        player.name as name
    FROM
        db_player as player
    JOIN 
        db_turn as turns ON turns.player_id = player.id
    GROUP BY
        turns.game_id,
        turns.player_id
    '''
query_2 = '''
    SELECT
        avg(number_of_turns),
        name
    FROM
        ({})
    GROUP BY
        name
'''.format(query_1)
run_query(
    query_2
)

[(14.35, 'BiggestFirst'), (27.4555, 'Replacement'), (14.6405, 'Tunnels')]